In [6]:
## Reload the modules each time there are modified
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [61]:
import numpy as np
import matplotlib.pyplot as plt

from src.arms import Bernoulli
from src.stochastic_bandit import MAB, BernoulliMAB
from src.multiplayers import multiplayer_env, PlayerRandTopUCB

In [63]:
Bernoulli(0.4)

Bernoulli(0.4)

In [64]:
MAB([Bernoulli(0.2), Bernoulli(0.3)], m=2), BernoulliMAB([0.2, 0.3])

(MAB([Bernoulli(0.2), Bernoulli(0.3)]), BernoulliMAB([0.2, 0.3]))

In [122]:
bandit = BernoulliMAB([0.2, 0.3, 0.5, 0.75, 0.8])
K, M = bandit.nb_arms, 3
players = [PlayerRandTopUCB(K, M) for _ in range(M)]

In [123]:
selections, collisions, sensing_infos = multiplayer_env(bandit, players, 44)

In [127]:
j = 1
L = [0] * K
for i in range(len(selections[0])):
    L[selections[j,i]] += sensing_infos[j,i]
L

[2.0, 0.0, 0.0, 24.0, 6.0]

In [128]:
players[j].cum_rewards

array([ 2.,  0.,  0., 24.,  6.])